In [1]:
import Random

Utility functions

In [2]:
function CDFsample(rng::Random.AbstractRNG, weights::Vector{Float64},n::Int, weightsum::Float64)
    t = rand(rng) * weightsum
    i = 1
    cuw = weights[1] #current cumulative sum
    while cuw < t && i < n #check if t in current part of weights, and you're in bounds
        i += 1
        @inbounds cuw += weights[i]
    end
    return i
end

function relative_error(x,y)
    abs(x-y)/y
end

function vectorreplace!(vec::Vector,index::UInt,newvalue)
    vec[index] = newvalue
    return vec
end

#To reduce allocations, you want to in place calc randvec and then take argmin
function exponential_sample!(rng::Random.AbstractRNG,weights::Vector{Float64},randvec::Vector{Float64})
    Random.rand!(rng,randvec)
    randvec .= -1 .* log.(randvec) ./ weights
    argmin(randvec)
end

function random_fill!(rng::Random.AbstractRNG,vec::Vector{Float64})
    for i in 1:length(a)
        vec[i] = rand(rng)
    end
    return nothing
end

random_fill! (generic function with 1 method)

Defining rates and system used

In [22]:
function constantrate(param::Float64)
    return param
end

function linearrate(param::Float64,moleculelevel::Int64)
    return param*moleculelevel
end

function hillfunction(lambda::Float64,n::Float64,K::Float64,x::Int64)
    if n >= 0
        return lambda*(x^n)/(K^n + x^n)
    else
        N = abs(n)
        return lambda*(K^N)/(K^N+x^N)
    end
end

function transcription_translation_rates!(state::Vector{Int64},params::Vector{Float64},rates::Vector{Float64})
    rates[1] = constantrate(params[1])
    rates[2] = linearrate(params[2],state[1])
    rates[3] = linearrate(params[3],state[1])
    rates[4] = linearrate(params[4],state[2])
    return nothing
end


transcription_translation_rates! (generic function with 1 method)

Data Storage Utilities

In [4]:
mutable struct online_weighted_Avg_Covs_2D
    meanx::Float64
    meany::Float64
    weightsum::Float64
    VX::Float64
    VY::Float64
    C::Float64
    function online_weighted_Avg_Covs_2D()
        return new(0,0,0,0,0,0)
    end

end

function updatestorage!(storage::online_weighted_Avg_Covs_2D,x,y,weight)
    storage.weightsum += weight
    dx = x - storage.meanx
    dy = y - storage.meany
    storage.meanx += (weight/storage.weightsum)*dx
    storage.meany += (weight/storage.weightsum)*dy
    storage.VX += weight*dx*(x-storage.meanx)
    storage.VY += weight*dy*(y-storage.meany)
    storage.C += weight*dx*(y-storage.meany)
    return nothing
end

updatestorage! (generic function with 1 method)

Main Code: Run a direct Gillespie simulation

In [17]:
function main(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]

    numreactions,statevars = size(outcomes)
    reactioncounters = fill(numsteps,numreactions)
    stoppingvec = reactioncounters .!= 0
    steps = 0
    rates = Vector{Float64}(undef,numreactions)
    zerorates = Vector{Bool}(undef,numreactions)

    calcstor = online_weighted_Avg_Covs_2D()
    while (steps <= maxsteps-1) & (any(stoppingvec))
        transcription_translation_rates!(statevector,params,rates)
        zerorates .= rates .== 0
        if all(zerorates)
            return "All rates trivially zero at state $(statevector) after $(steps) steps"
        end
        totalrate = sum(rates)
        reaction = CDFsample(rng,rates,numreactions,totalrate)
        timestep = log(1/rand(rng))/totalrate
        delta = view(outcomes,reaction,:)
        statevector .+= delta
        updatestorage!(calcstor,statevector[1],statevector[2],timestep)
        steps += 1
        reactioncounters[reaction] -= 1
        if reactioncounters[reaction] <= 0
            reactioncounters[reaction] = 0
            stoppingvec[reaction] = false
        end
    end
    expectedmeanm = params[1]/params[2]
    expectedmeanp = expectedmeanm*params[3]/params[4]
    expectedetamm = 1/expectedmeanm
    expectedcov = expectedetamm*(params[4]/(params[4]+params[2]))
    expectedetapp = 1/expectedmeanp + expectedcov

    calcdmeanm = calcstor.meanx
    calcdmeanp = calcstor.meany
    calcdetamm = (calcstor.VX/calcstor.weightsum)/calcdmeanm^2
    calcdetapp = (calcstor.VY/calcstor.weightsum)/calcdmeanp^2
    calcdcovmp = (calcstor.C/calcstor.weightsum)/(calcdmeanm*calcdmeanp)

    println(relative_error(calcdmeanm,expectedmeanm))
    println(relative_error(calcdmeanp,expectedmeanp))
    println(relative_error(calcdcovmp,expectedcov))
    println(relative_error(calcdetamm,expectedetamm))
    println(relative_error(calcdetapp,expectedetapp))
    println(calcdetapp)
    println(steps)

    return nothing
end

main (generic function with 1 method)

The test below verifies this system can return results for the analytic case that are sufficiently accurate. The exact value of $\eta_{pp}$ here is 0.14

In [21]:
main([1000,1000],[5.,1.,5.,1.],10^8,10^10,Random.MersenneTwister())

0.0030727892466691385
0.0006354904909926518
0.0021944623985092893
0.006258558449920293
0.00915483480317233
0.14128167687244414
1200086342


Next, we want to create a test showing that the flux balance and covariance balance equations are satisfied. 

The flux balance equations require that for all species $\langle R^+_i \rangle = \langle R^-_i \rangle$.

The covariance balance equations require that for all pairs of species 
$$ Cov\left[x_i,R^-_j - R^+_j\right] + Cov\left[x_j,R^-_i - R^+_i\right] = \langle\Sigma^N_{k=1}\delta_{ik}\delta_{jk}r_k\rangle$$

On the right hand side, the sum requires going through all reactions and checking checking how much $ x_i,x_j$ change in the $k$th reaction. Therefore, in the case like mRNA and protein where only one species changes at a time with step size, this is the average total rate $\sum \langle r_k \rangle$ where the species changes if $i=j$ and otherwise 0. 

Rewrite to avoid 0=0 comparisons:
$$ Cov\left[x_i,R^-_j\right] + Cov\left[x_j,R^-_i\right] = \langle\Sigma^N_{k=1}\delta_{ik}\delta_{jk}r_k\rangle + \left(Cov\left[x_i, R^+_j\right] + Cov\left[x_j, R^+_i\right]\right)$$

We will reimplment a Gillespie algorithm, but now with a modified rate set that includes protein feedback such that $R_m^+ = \frac{p^n}{K^n+p^n}$. We will check whether the flux balance and covariance balance equations are satisfied after a large number of steps. We will test it on a system displaying negative feedback.

In [52]:
function cov_balance_comp(CiRmj,CjRmi,Dij,CiRpj,CjRpi)
    LHS = CiRmj+CjRmi
    RHS = CiRpj+CjRpi+Dij
    return relative_error(RHS,LHS)
end

function tt_protein_fb_rates!(state::Vector{Int64},params::Vector{Float64},rates::Vector{Float64})
    rates[1] = hillfunction(params[1],params[5],params[6],state[2])
    rates[2] = linearrate(params[2],state[1])
    rates[3] = linearrate(params[3],state[1])
    rates[4] = linearrate(params[4],state[2])
    return nothing
end

mutable struct BalanceEqData
    meanx::Float64
    meany::Float64
    weightsum::Float64
    VX::Float64
    VY::Float64
    CXY::Float64
    meanRpX::Float64
    meanRmX::Float64
    meanRpY::Float64
    meanRmY::Float64
    CXRmX::Float64
    CXRpX::Float64
    CYRmX::Float64
    CYRpX::Float64
    CXRmY::Float64
    CXRpY::Float64
    CYRmY::Float64
    CYRpY::Float64

    function BalanceEqData()
        return new(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    end
    
end

function updatestorage!(storage::BalanceEqData,state::Vector{Int64},rates::Vector{Float64},weight)
    #States: [x,y]
    #Rates: [RpX,RmX,RpY,RmY]
    storage.weightsum += weight
    rw = weight/storage.weightsum

    x = state[1]
    y = state[2]
    RpX = rates[1]
    RmX = rates[2]
    RpY = rates[3]
    RmY = rates[4]


    dx = x - storage.meanx
    dy = y - storage.meany
    dRpX = RpX - storage.meanRpX
    dRmX = RmX - storage.meanRmX
    dRpY = RpY - storage.meanRpY
    dRmY = RmY - storage.meanRmY

    storage.meanx += (rw)*dx
    storage.meany += (rw)*dy
    storage.meanRpX += (rw)*dRpX
    storage.meanRmX += (rw)*dRmX
    storage.meanRpY += (rw)*dRpY
    storage.meanRmY += (rw)*dRmY

    xdiff = x - storage.meanx
    ydiff = y - storage.meany

    storage.VX += weight*dx*xdiff
    storage.VY += weight*dy*ydiff

    storage.CXY += weight*dx*ydiff

    storage.CXRmX += weight*dRmX*xdiff
    storage.CXRpX += weight*dRpX*xdiff
    storage.CYRmX += weight*dRmX*ydiff
    storage.CYRpX += weight*dRpX*ydiff
    storage.CXRmY += weight*dRmY*xdiff
    storage.CXRpY += weight*dRpY*xdiff
    storage.CYRmY += weight*dRmY*ydiff
    storage.CYRpY += weight*dRpY*ydiff
    return nothing
end

updatestorage! (generic function with 2 methods)

In [53]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]

    numreactions,statevars = size(outcomes)
    reactioncounters = fill(numsteps,numreactions)
    stoppingvec = reactioncounters .!= 0
    steps = 0
    rates = Vector{Float64}(undef,numreactions)
    zerorates = Vector{Bool}(undef,numreactions)

    calcstor = BalanceEqData()
    while (steps <= maxsteps-1) & (any(stoppingvec))
        tt_protein_fb_rates!(statevector,params,rates)
        zerorates .= rates .== 0
        if all(zerorates)
            return "All rates trivially zero at state $(statevector) after $(steps) steps"
        end
        totalrate = sum(rates)
        reaction = CDFsample(rng,rates,numreactions,totalrate)
        timestep = log(1/rand(rng))/totalrate
        delta = view(outcomes,reaction,:)
        statevector .+= delta
        updatestorage!(calcstor,statevector,rates,timestep)
        steps += 1
        reactioncounters[reaction] -= 1
        if reactioncounters[reaction] <= 0
            reactioncounters[reaction] = 0
            stoppingvec[reaction] = false
        end
    end

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    println(steps)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [54]:
calcs = balance_test([1000,1000],[10.,1.,5.,1.,-1.,200.],10^8,10^10,Random.MersenneTwister())

Flux Balance on x
0.00017966070031279695
Flux Balance on y
2.267653427449925e-5
Cov Balance on x,x
0.007353657857875004
Cov Balance on y,y
0.011714656476694875
Cov Balance on y,x
0.015444923655552873
1200386289
8.308762163490087
41.49480216553076


BalanceEqData(8.308762163490087, 41.49480216553076, 1.205129650045403e7, 9.336600039407799e7, 1.5434102394243867e9, 2.081844391692029e8, 8.299296209677898, 8.30078753497958, 41.50393767476148, 41.504878861569544, 9.241556300162306e7, -6.931110136937372e6, 2.1117274537703872e8, -5.190972748046623e7, 2.0542973566527665e8, 4.6207781500836915e8, 1.53802818516003e9, 1.0558637268850957e9)

37.85979874291953

-0.18839068850767732